In [2]:
from sklearn import datasets

wine_dataset = datasets.load_wine()
X = wine_dataset.data
y = wine_dataset.target

In [3]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [4]:
from skopt import BayesSearchCV

n_iterations = 50

In [5]:
estimator = xgb.XGBClassifier(
    n_jobs=-1,
    objective="multi:softmax",
    eval_metric="merror",
    verbosity=0,
    num_class=len(set(y)),
)

In [6]:
search_space = {
    "learning_rate": (0.01, 1.0, "log-uniform"),
    "min_child_weight": (0, 10),
    "max_depth": (1, 50),
    "max_delta_step": (0, 10),
    "subsample": (0.01, 1.0, "uniform"),
    "colsample_bytree": (0.01, 1.0, "log-uniform"),
    "colsample_bylevel": (0.01, 1.0, "log-uniform"),
    "reg_lambda": (1e-9, 1000, "log-uniform"),
    "reg_alpha": (1e-9, 1.0, "log-uniform"),
    "gamma": (1e-9, 0.5, "log-uniform"),
    "min_child_weight": (0, 5),
    "n_estimators": (5, 5000),
    "scale_pos_weight": (1e-6, 500, "log-uniform"),
}

In [7]:
cv = StratifiedKFold(n_splits=3, shuffle=True)

In [8]:
bayes_cv_tuner = BayesSearchCV(
    estimator=estimator,
    search_spaces=search_space,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1,
    n_iter=n_iterations,
    verbose=0,
    refit=True,
)

In [11]:
import pandas as pd
import numpy as np


def print_status(optimal_result):
    """Shows the best parameters found and accuracy attained of the search so far."""
    models_tested = pd.DataFrame(bayes_cv_tuner.cv_results_)
    best_parameters_so_far = pd.Series(bayes_cv_tuner.best_params_)
    print(
        "Model #{}\nBest accuracy so far: {}\nBest parameters so far: {}\n".format(
            len(models_tested),
            np.round(bayes_cv_tuner.best_score_, 3),
            bayes_cv_tuner.best_params_,
        )
    )

    clf_type = bayes_cv_tuner.estimator.__class__.__name__
    models_tested.to_csv(clf_type + "_cv_results_summary.csv")

In [10]:
result = bayes_cv_tuner.fit(X, y, callback=print_status)

Model #1
Best accuracy so far: 0.961
Best parameters so far: {'colsample_bylevel': 0.010150465316599811, 'colsample_bytree': 0.012849506210578468, 'gamma': 3.8683966687522224e-08, 'learning_rate': 0.023057910501188637, 'max_delta_step': 4, 'max_depth': 21, 'min_child_weight': 4, 'n_estimators': 2271, 'reg_alpha': 0.032703008308886596, 'reg_lambda': 6.659896357200116e-05, 'scale_pos_weight': 1.1370073471031226e-06, 'subsample': 0.49179261256464213}

Model #2
Best accuracy so far: 0.961
Best parameters so far: {'colsample_bylevel': 0.010150465316599811, 'colsample_bytree': 0.012849506210578468, 'gamma': 3.8683966687522224e-08, 'learning_rate': 0.023057910501188637, 'max_delta_step': 4, 'max_depth': 21, 'min_child_weight': 4, 'n_estimators': 2271, 'reg_alpha': 0.032703008308886596, 'reg_lambda': 6.659896357200116e-05, 'scale_pos_weight': 1.1370073471031226e-06, 'subsample': 0.49179261256464213}

Model #3
Best accuracy so far: 0.972
Best parameters so far: {'colsample_bylevel': 0.651519337